## 팀프로젝트4


### 라이브러리

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from distutils.version import LooseVersion
from sqlalchemy import create_engine, inspect
from google.cloud import storage, bigquery


### data 파일 용량 확인

In [ ]:
# 파일 용량 확인
def get_csv_file_sizes(directory):
    file_sizes = {}
    for filename in os.listdir(directory):
        if filename.endswith('.parquet'):
            file_path = os.path.join(directory, filename)
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = file_size_bytes / (1024 * 1024)
            file_sizes[filename] = file_size_mb
    return file_sizes

In [ ]:
# directory = '데이터가 저장된 위치'
directory = './origin_data'
csv_file_sizes = get_csv_file_sizes(directory)
sorted_dict = dict(sorted(csv_file_sizes.items(), key=lambda item: item[1], reverse=True))
for filename, size in sorted_dict.items():
    print(f'{filename:<50}: {size:.2f} MB')

### GCS로 부터 parquet파일 받아오기

In [ ]:
# 환경 변수에 JSON 키 파일 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./config/codeit-project-567b5092fd38.json"

# GCS 클라이언트 및 BigQuery 클라이언트 초기화
storage_client = storage.Client()
bigquery_client = bigquery.Client()

def download_parquet_from_gcs(bucket_name, prefix):
    """
    GCS에서 Parquet 파일 다운로드 및 병합.
    :param bucket_name: GCS 버킷 이름
    :param prefix: 다운로드할 경로 (GCS 버킷 내부 폴더)
    :return: 파일별 데이터프레임 딕셔너리 (key: 파일 이름, value: 데이터프레임)
    """
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)  # 지정된 경로의 파일 검색
    dataframes = {}  # 파일별 데이터프레임 저장
    file_names = []

    for blob in blobs:
        if blob.name.endswith(".parquet"):
            file_name = blob.name.split("/")[-1].replace(".parquet", "")  # 파일 이름 추출
            print(f"Downloading: {blob.name}")
            # GCS에서 바로 메모리로 읽기
            with blob.open("rb") as file:
                df = pd.read_parquet(file, engine="pyarrow")
                dataframes[file_name] = df
                file_names.append(file_name)

    if not dataframes:
        print(f"No Parquet files found at prefix: {prefix}")
    return dataframes, file_names  # 파일 이름별 데이터프레임 딕셔너리 반환


In [ ]:
## 그냥 데이터 다운로드만 하고 확인 할 때 사용.
## 실행, prefix, dataset_name은 동일하게 들어갈 것 같습니다.
if __name__ == "__main__":
    db_name = "origin/hackle"
    bucket_name = "finalproject_sprint"
    prefix = db_name  # GCS 내부의 특정 경로(버킷에서 파일이 저장된 폴더 이름.)
    # dataset_name = db_name  # 데이터셋 이름 지정

    # GCS에서 Parquet 데이터 다운로드
    dataframes, file_names = download_parquet_from_gcs(bucket_name, prefix)
    print()
    print(f"DB name : {db_name}, \ntable_name : {file_names}")

    # GCS에서 받은 parquet의 file이름으로 데이터 저장
    for file_name in file_names:
        globals()[file_name] = dataframes[f'{file_name}']

In [ ]:
# 저장 경로 설정
output_dir = "./modified_data"
os.makedirs(output_dir, exist_ok=True)  # 저장 디렉토리 생성

### device_properties

In [ ]:
display(device_properties)

In [ ]:
device_properties['device_vendor'].unique(), device_properties['device_vendor'].nunique()

In [ ]:
device_properties['device_vendor'] = device_properties['device_vendor'].str.upper()

In [ ]:
device_properties['device_vendor'].unique(), device_properties['device_vendor'].nunique()

In [ ]:
device_properties.to_parquet('modified_data/hackle_device_properties.parquet')

### user_properties

In [ ]:
user_properties

In [ ]:
user_properties[user_properties['user_id'].isna()]

In [ ]:
# 숫자가 아닌 값들만 필터링하는 코드
non_numeric_ids = user_properties[user_properties['user_id'].apply(lambda x: not str(x).isdigit())]

# 결과 확인
print(non_numeric_ids)

## 해당부분을 확인한 결과 df내부에 존재한것으로 확인됨.

In [ ]:
import seaborn as sns

sns.countplot(user_properties['gender'])
plt.show()

In [ ]:
user_properties['grade'] = user_properties['grade'].astype('category')

In [ ]:
sns.countplot(user_properties['grade'])
plt.show()

In [ ]:
user_properties.to_parquet('data1.parquet')

### hackle_properties

In [ ]:
hackle_properties

In [ ]:
hackle_properties['session_id'].value_counts()

In [ ]:
# value_counts()를 사용하여 각 값의 개수 계산
value_counts = hackle_properties['session_id'].value_counts()

# 값이 2 이상인 것만 필터링
filtered_counts = value_counts[value_counts >= 2]

# 결과 확인
print(filtered_counts)


In [ ]:
# 필터링된 결과를 데이터프레임으로 변환
## 이부분은 versionname이 가장 높은 버전을 기준으로 데이터를 slicing
## 기존의 versionname이 낮은 데이터를 삭제.
filtered_df = hackle_properties[hackle_properties['session_id'].isin(filtered_counts.index)]
filtered_df['versionname'] = filtered_df['versionname'].apply(LooseVersion)

filtered_df2 = hackle_properties[~hackle_properties['session_id'].isin(filtered_counts.index)]

latest_version = filtered_df.loc[filtered_df.groupby('session_id')['versionname'].idxmax()] ## 디테일하게 할 꺼면 수정! 
latest_version['versionname'] = latest_version['versionname'].astype(str)

In [ ]:
filtered_df2.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df2

In [ ]:
none_user_id = filtered_df2[filtered_df2['user_id'].isin([''])]['session_id'].tolist()

user_properties[user_properties['user_id'].isin(none_user_id)]

In [ ]:
filtered_df2[filtered_df2['user_id'].isin(['', ' ', 'null', 'None', None, pd.NA])]

In [ ]:
latest_version.isnull().count()

In [ ]:
total_df = pd.concat([latest_version, filtered_df2], axis=0)
total_df

In [ ]:
# 결과 확인
# filtered_hackle_properties
missing_values = latest_version[latest_version.isnull().any(axis=1)]

missing_values

In [ ]:
total_df.to_parquet('modified_data/hackle_hackle_properties.parquet')

In [ ]:
latest_version

### hackle_events

In [ ]:
display(hackle_events.head())
hackle_events.shape

In [ ]:
hackle_events[hackle_events['event_datetime'].isin(['', ' ', 'null', 'None', None, pd.NA])]

In [ ]:
hackle_events[hackle_events['session_id'].isin(['', ' ', 'null', 'None', None, pd.NA])]

In [ ]:
hackle_events.head()

In [ ]:
hackle_events[hackle_events['session_id'].str.strip().isna()]

In [ ]:
hackle_events.dtypes

In [ ]:
hackle_events.to_parquet("modified_data/hackle_hackle_events.parquet")

### master table

- hackle_events table을 기준으로 참고가능한 columns을 붙여서 master table 생성

In [ ]:
hackle_events.head()

In [ ]:
hackle_events.shape

In [ ]:
hackle_events.drop(columns='id', inplace=True)

In [ ]:
merge_df = hackle_events.merge(total_df, how='left', on='session_id')
merge_df.drop(columns='id', inplace=True)

In [ ]:
merge_df = merge_df.merge(device_properties, how='left', on='device_id')
merge_df.drop(columns='id', inplace=True)

In [ ]:
merge_df.head()

In [ ]:
## hackle_events datetime 정렬
hackle_events = hackle_events.sort_values(by='event_datetime')

In [ ]:
hackle_events['event_key'].unique()

In [ ]:
hackle_events[hackle_events['event_key'] == 'click_question_ask']

#### DAU

In [ ]:
hackle_events["event_date"] = hackle_events["event_datetime"].dt.date

# DAU 계산
dau = hackle_events.groupby("event_date")["session_id"].nunique()
print("DAU:\n", dau)

In [ ]:
# 시각화
plt.figure(figsize=(12, 6))
dau.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Daily Active Users (DAU)", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Unique Users", fontsize=12)
plt.xticks(rotation=90)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

#### WAU


In [ ]:
hackle_events['week'] = hackle_events['event_datetime'].dt.strftime("%Y-%U")
wau = hackle_events.groupby("week")['session_id'].nunique()
print("WAU: \n", wau)

In [ ]:
# 시각화
plt.figure(figsize=(12, 6))
wau.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Weekly Active Users (WAU)", fontsize=16)
plt.xlabel("Week", fontsize=12)
plt.ylabel("Unique Users", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

#### Cohort(weekly)

In [ ]:
hackle_events['event_date'] = pd.to_datetime(hackle_events['event_date'], errors="coerce")

#### Cohort(daily)

In [ ]:
## 코호트 계산을 위해서 처음 접속 날짜 계산
hackle_events['first_date'] = hackle_events.groupby('session_id')['event_datetime'].transform("min").dt.date
hackle_events['first_date'] = pd.to_datetime(hackle_events['first_date'])

## 접속한 주차 계산(1주, 2주, 3주..)
hackle_events['cohort_day'] = (hackle_events['event_date'] - hackle_events['first_date']).dt.days

## Cohort Retention table create
cohort = hackle_events.groupby(['first_date', 'cohort_day'])['session_id'].nunique().unstack(1)

## chort retension ratio
cohort_retention_daily = cohort.divide(cohort.iloc[:, 0], axis=0) * 100

cohort_retention_daily.index = cohort_retention_daily.index.strftime("%Y-%m-%d")

print(cohort_retention_daily)

In [ ]:
plt.figure(figsize=(24, 20))
sns.heatmap(cohort_retention_daily, annot=True, fmt='.1f', cmap='Reds', cbar_kws={"label": "Retention Rate (%)"})
plt.title("Cohort Retention Analysis(Daily)", fontsize=16)
plt.xlabel("Cohort Day", fontsize=12)
plt.ylabel("First Active Date", fontsize=12)
plt.show()

#### Cohort(Weekly)

In [ ]:
## 코호트 계산을 위해서 처음 접속 날짜 계산
hackle_events['first_date'] = hackle_events.groupby('session_id')['event_datetime'].transform("min").dt.date

# 첫 접속이 속한 주의 첫 날짜를 기준으로 주차를 계산
hackle_events["first_week_start"] = pd.to_datetime(hackle_events["first_date"]).dt.to_period("W").dt.start_time

## 접속한 주차 계산(1주, 2주, 3주..)
hackle_events["cohort_week"] = (hackle_events["event_date"].dt.to_period("W").dt.start_time - hackle_events["first_week_start"]).dt.days // 7

## Cohort Retention table create
cohort = hackle_events.groupby(['first_week_start', 'cohort_week'])['session_id'].nunique().unstack(1)

## chort retension ratio
cohort_retention_weekly = cohort.divide(cohort.iloc[:, 0], axis=0) * 100

cohort_retention_weekly.index = cohort_retention_weekly.index.strftime("%Y-%m-%d")

print("주 단위 Cohort Retention:")
print(cohort_retention_weekly)

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(cohort_retention_weekly, annot=True, fmt='.1f', cmap='Reds', cbar_kws={"label": "Retention Rate (%)"})
plt.title("Cohort Retention Analysis(Weekly)", fontsize=16)
plt.xlabel("Cohort Week", fontsize=12)
plt.ylabel("First Active Date", fontsize=12)
plt.show()

#### engagement

In [ ]:
# 일별 평균 이벤트 수 계산
engagement = hackle_events.groupby("event_date")["event_key"].count() / hackle_events.groupby("event_date")["event_key"].nunique()

# 시각화
plt.figure(figsize=(10, 6))
engagement.plot(kind="line", marker="o", color="green")
plt.title("User Engagement (Average Events per User)", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Average Events per User", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

#### Aoripi

In [ ]:
hackle_events.head()

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import random


df = hackle_events[['session_id', 'event_datetime', 'event_key']]

session_ids = df['session_id'].unique()
sample_size = int(len(session_ids) * 0.1) # 샘플 비율 조정
sampled_session_ids = random.sample(session_ids.tolist(), sample_size)
df_sampled = df[df['session_id'].isin(sampled_session_ids)]

print("샘플링된 데이터 크기:", len(df_sampled))

user_events_sampled = df_sampled.groupby('session_id')['event_key'].apply(list).tolist()

te = TransactionEncoder()
te_ary_sampled = te.fit(user_events_sampled).transform(user_events_sampled)
df_encoded_sampled = pd.DataFrame(te_ary_sampled, columns=te.columns_)

frequent_itemsets_sampled = apriori(df_encoded_sampled, min_support=0.1, use_colnames=True) # min_support 조정

print(frequent_itemsets_sampled)
if frequent_itemsets_sampled.empty:
    print("frequent_itemsets_sampled가 비어 있습니다. 데이터 또는 min_support 값을 확인하세요.")
    print("df_encoded_sampled shape:", df_encoded_sampled.shape)
    print(df_encoded_sampled.head())
    exit()

rules_sampled = association_rules(frequent_itemsets_sampled, metric="confidence", min_threshold=0.5)
rules_lift_sampled = association_rules(frequent_itemsets_sampled, metric="lift", min_threshold=1)

print("\n샘플 데이터의 빈발 항목 집합:\n",frequent_itemsets_sampled)
print("\n샘플 데이터의 연관 규칙:\n", rules_sampled)
print("\n샘플 데이터의 향상도 기반 규칙:\n",rules_lift_sampled)

#### event_key pie chart

In [ ]:
counts = hackle_events['event_key'].value_counts()
ratios = (counts / hackle_events.shape[0]) * 100

# 비율이 3% 미만인 이벤트들을 "others"로 합산
others_ratio = ratios[ratios < 3].sum()

# "others" 항목 생성
ratios = ratios[ratios >= 3]  # 3% 이상인 비율만 남기고
ratios["other_events"] = others_ratio  # "others" 항목 추가

# 결과 출력
print(ratios)
ratios_df = pd.DataFrame(ratios).reset_index(level=0)
ratios_df.to_csv('ratios.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 원형 차트 그리기
plt.figure(figsize=(9, 9))  # 차트 크기 설정
plt.pie(
    ratios, 
    labels=ratios.index,  # 라벨 설정
    autopct='%1.1f%%',    # 퍼센트 표시
    startangle=90,        # 시작 각도
    colors=plt.cm.tab10.colors  # 색상 팔레트 선택 (옵션)
)

plt.title('Ratio(%) of Each Events')  # 제목 설정
plt.tight_layout()
plt.show()